### Setting up the environment 

In [ ]:
from sqlalchemy import create_engine
import pandas as pd

# Connection parameters
PGHOST = "rammyserver.postgres.database.azure.com"
PGUSER = "rammysubiate"
PGPASSWORD = "data11212*" 
PGPORT = 5432
PGDATABASE = "postgres"

# Create engine
engine = create_engine(
    f"postgresql+psycopg2://{PGUSER}:{PGPASSWORD}@{PGHOST}:{PGPORT}/{PGDATABASE}"
)


### Reusable SQL Query Function in Python

In [59]:
def run_query(query):
    """
    Executes a SQL query using the provided SQLAlchemy engine and returns a DataFrame.
    
    Parameters:
    - query (str): The SQL query to execute.
    - engine : SQLAlchemy engine connected to DB.
    
    Returns:
    - pd.DataFrame: Result of the query.
    """
    try:
        df = pd.read_sql(query, engine)
        return df
    except Exception as e:
        print(f"Error running query: {e}")
        return None


In [63]:
query = """

SELECT *
FROM fact_sales
LIMIT 5;

"""

df = run_query(query)
df

,transaction_num,date,product_num,product_name,price,quantity,revenue,customer_num,country,customer_frequency,customer_contribution
0,559676,2019-07-11,23089,glass bon bon jar,11.94,12,143.28,17340,United Kingdom,most_frequent,most_contributor
1,559676,2019-07-11,22082,ribbon reel stripes design,11.94,10,119.40,17340,United Kingdom,most_frequent,most_contributor
2,559676,2019-07-11,22090,paper bunting retrospot,13.27,6,79.62,17340,United Kingdom,most_frequent,most_contributor
3,559676,2019-07-11,20750,red retrospot mini cases,18.40,2,36.80,17340,United Kingdom,most_frequent,most_contributor
4,559676,2019-07-11,23192,bundle of 3 alphabet exercise books,11.94,12,143.28,17340,United Kingdom,most_frequent,most_contributor


### Checking Column Names and Data Types

In [66]:
query = """
SELECT column_name, data_type
FROM INFORMATION_SCHEMA.COLUMNS
WHERE table_name = 'fact_sales'
ORDER BY ordinal_position

"""
data_check = run_query(query)
data_check


,column_name,data_type
0,transaction_num,text
1,date,timestamp without time zone
2,product_num,text
3,product_name,text
4,price,double precision
5,quantity,bigint
6,revenue,double precision
7,customer_num,bigint
8,country,text
9,customer_frequency,text


### Row Count

In [62]:
query = """

SELECT COUNT(*) AS row_count
FROM fact_sales

"""
row_count = run_query(query)
row_count


,row_count
0,522601


### Checking for NULLs

In [64]:
query = """

SELECT COUNT(*) - COUNT(transaction_num) AS transaction_num_nulls,
    COUNT(*) - COUNT(product_num) AS product_num_nulls,
    COUNT(*) - COUNT(product_name) AS product_name_nulls,
    COUNT(*) - COUNT(price) AS price_nulls,
    COUNT(*) - COUNT(quantity) AS quantity_nulls
FROM fact_sales

"""

null_check = run_query(query)
null_check

,transaction_num_nulls,product_num_nulls,product_name_nulls,price_nulls,quantity_nulls
0,0,0,0,0,0


### Checking for Duplicates

In [ ]:
columns = ["transaction_num","date","product_num","product_name","price",
           "quantity","revenue","customer_num","country",
           "customer_frequency","customer_contribution"]

cols = (", ").join(columns)
query = f"""
WITH ranked AS (
    SELECT *,
        ROW_NUMBER() OVER(PARTITION BY {cols} ORDER BY date) AS rank
    FROM fact_sales
)
SELECT COUNT(*) AS duplicate_count
FROM ranked
WHERE rank > 1;
"""

dup_check = run_query(query)
dup_check

,duplicate_count
0,0
